In [4]:
using Pkg
# Pkg.activate(".")
# Pkg.instantiate()
# Pkg.add("SQLite")
# Pkg.add("DBInterface")
# Pkg.add("JSON3")
# Pkg.add("EasyConfig")
# Pkg.add("MurmurHash3")
# Pkg.add("TextAnalysis")
# Pkg.add("UUIDs")
# Pkg.add("PooledArrays")
# Pkg.add("HTTP")
# Pkg.add("EasyConfig")

# include("src/lisa_sets.jl")
include("src/lisa_neo4j.jl")
include("src/lisa_graph.jl")

using SQLite, DBInterface
using MurmurHash3
using TextAnalysis
using HTTP, JSON3
using Base64
using PooledArrays
using EasyConfig

db = Graph.DB("lisa.db")
# Define the URL of your Neo4j instance
url = "http://localhost:7474/db/neo4j/tx/commit"
# Define the headers
headers = ["Content-Type" => "application/json", "Authorization" => "Basic " * base64encode("neo4j:neo4j")]

2-element Vector{Pair{String, String}}:
  "Content-Type" => "application/json"
 "Authorization" => "Basic bmVvNGo6bmVvNGo="

In [2]:
# Search for all the node's references (node's sha1) with the name "AVE" and "name" as the property
rows = LisaNeo4j.search_by_tokens(db.sqlitedb, "ave", "name", "danilo")
# Collect all the node's references from edges table
edges = Vector()
edges_refs = LisaNeo4j.select_edges(db.sqlitedb, rows, edges)
# Collect all the nodes from search request and the nodes from the edges
refs = union(edges_refs,rows)
# Collect the actual nodes
nodes = Vector()
LisaNeo4j.select_nodes(db.sqlitedb, refs, nodes)
# Add nodes to the Neo4j database
for node in nodes
    labels = replace(string(node.labels), ";" => "")
    query = LisaNeo4j.add_neo4j_node(labels, node)
    data = LisaNeo4j.request(url, headers, query)
    # println(data)
end
# Add edges to the Neo4j database
for edge in edges
    # println(edge)
    query = LisaNeo4j.add_neo4j_edge(edge)
    data = LisaNeo4j.request(url, headers, query)
    # println(data)
end

In [3]:
"""
    In this cell we are going to to generate relations between matched csv files (nodes with labes "csv_file")
"""
# Define your Cypher query
query = LisaNeo4j.cypher("MATCH (n:csv_file) RETURN n.labels, n.sha1, n.d_sha1, n.dataset, n.props LIMIT 20")
# Parse the response
json = LisaNeo4j.request(url, headers, query)
hlls = Dict{String, LisaNeo4j.Neo_node}()
LisaNeo4j.collect_hll_sets(json, hlls)
# Lets see what we get
for (k, v) in hlls
    # println("sha1: ", k, " hll: ", SetCore.count(v.hll_set), v)
end
# Generate the relations between match csv files
for (k, v) in hlls
    for (k1, v1) in hlls
        if k != k1
            match = SetCore.jaccard(v.hll_set, v1.hll_set)
            if match > 50
                props = JSON3.write(Dict("similarity" => match))
                dict = Dict{String, Any}("source" => k, "target" => k1, "r_type" => "match", "props" => props)
                df_row = LisaNeo4j.dict_to_dfrow(dict)
                query = LisaNeo4j.add_neo4j_edge(df_row)
                data = LisaNeo4j.request(url, headers, query)
                # println(data)
            end
        end
    end
end